# Assignment 1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to download HTML pages from a website?
* How to extract relevant content from an HTML page? 

Furthermore, you will gain a deeper understanding of the data science lifecycle.

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) rather than [lxml](http://lxml.de/) to parse an HTML page and extract data from the page.

3. Please follow python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of this topic. I found that this is a good resource: [Tutorial: Web Scraping and BeautifulSoup](https://www.dataquest.io/blog/web-scraping-beautifulsoup/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. Your job is to extract insights from SFU data to answer questions. 

In this assignment, you will do two tasks. Please recall the high-level data science lifecycle below. I suggest that when doing this assignment, please remind yourself of what data you collected and what questions you tried to answer.


<img src="lifecycle.png" width="500">
<center><h4>Figure 1. Data Science Lifecycle</h4></center>

## Task 1: SFU CS Faculty Members

Sometimes you don't know what questions to ask. No worries. Start collecting data first. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [https://www.sfu.ca/computing/people/faculty.html](https://www.sfu.ca/computing/people/faculty.html).




### (a) Crawl Web Page

A web page is essentially a file stored in a remote machine (called web server). Please write code to download the HTML page and save it as a text file ("csfaculty.html").

In [1]:
from requests import get
url = 'https://www.sfu.ca/computing/people/faculty.html'
response = get(url)
with open('csfaculty.html', 'w') as file:
    file.write(response.text)

### (b) Extract Structured Data

Please write code to extract relevant content (name, rank, area, profile, homepage) from "csfaculty.html" and save them as a CSV file (like [faculty_table.csv](./faculty_table.csv)). 

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
names = []
ranks = []
areas = []
profiles = []
homepages = []

with open('csfaculty.html', 'r') as file:
    contents = file.read()
    soup = BeautifulSoup(contents, 'lxml')
members = soup.find_all('div', class_="textimage section")
members = members[:-1]  # remove the last member
for member in members:
    name_rank = member.h4.text.split(', ')
    name = name_rank[0]
    rank = name_rank[1].replace(u'\xa0', '').replace('\n', '')
    if 'Area:' in rank:
        temp = rank.split('Area:')
        rank = temp[0]
        area = temp[1]
    else:
        area = member.p.text.replace('Area', '').replace(
            ':', '').strip()
    links = member.find_all('a', href=True)
    profile = ''
    homepage = ''
    for l in links:
        if 'Profile' in l.text:
            if 'http://www.sfu.ca' in l['href']:
                profile = l['href']
            else:
                profile = 'http://www.sfu.ca'+l['href']
        elif l.text == 'Home Page':
            homepage = l['href']
    names.append(name)
    ranks.append(rank)
    areas.append(area)
    profiles.append(profile)
    homepages.append(homepage)
df = pd.DataFrame({'name': names,
                   'rank': ranks,
                   'area': areas,
                   'profile': profiles,
                   'homepage': homepages
                   })
df.to_csv('faculty_table_test.csv', index=False)

IndexError: list index out of range

### (c) Interesting Finding

Note that you don't need to do anything for Task 1(c). The purpose of this part is to give you some sense about how to leverage exploratory data analysis (EDA) to come up with interesting questions about the data. EDA is an important topic in data science; you will  learn it soon from this course. 


First, please install [dataprep](http://dataprep.ai).
Then, run the cell below. 
It shows a bar chart for every column. What intersting findings can you get from these visualizations? 

In [1]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_table.csv")
plot(df)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/holoviews/plotting/util.py:685: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  [cmap for cmap in cm.cmap_d if not


Below are some examples:

**Finding 1:** Professor# (29) is more than 3x larger than Associate Professor# (9). 

**Questions:** Why did it happen? Is it common in all CS schools in Canada? Will the gap go larger or smaller in five years? What actions can be taken to enlarge/shrink the gap?


**Finding 2:** Homepage has 20.3% missing values. 

**Questions:** Why are there so many missing values? Is it because many faculty do not have their own homepages or do not add their homepages to the school page? What actions can be taken to avoid this to happen in the future? 

## Task 2: Age Follows Normal Distribution?

In this task, you start with a question and then figure out what data to collect.

The question that you are intersted in is `Does SFU CS faculty age follow a normal distribution?`

To estimate the age of a faculty member, you can collect the year in which s/he graduates from a university (`gradyear`) and then estimate `age` using the following equation:

$$age \approx 2020+23 - gradyear$$

For example, if one graduates from a university in 1990, then the age is estimated as 2020+23-1990 = 53. 



### (a) Crawl Web Page

You notice that faculty profile pages contain graduation information. For example, you can see that Dr. Jiannan Wang graduated from Harbin Institute of Technology in 2008 at [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please write code to download the 64 profile pages and save each page as a text file. 

In [ ]:
import pandas as pd
from requests import get
from time import sleep
from random import randint
import os
dirs = 'members/'
if not os.path.exists(dirs):
    os.makedirs(dirs)
df = pd.read_csv('faculty_table_test.csv')
for index, row in df.iterrows():
    with open(dirs+row['name']+'.txt', 'w') as file:
        response = get(row['profile'])
        file.write(response.text)
        sleep(randint(1, 3))

### (b) Extract Structured Data

Please write code to extract the earliest graduation year (e.g., 2008 for Dr. Jiannan Wang) from each profile page, and create a csv file like [faculty_grad_year.csv](./faculty_grad_year.csv). 

In [ ]:
from bs4 import BeautifulSoup
import re
import os
pattern = re.compile('\d{4}')
dirs = 'members/'
files = os.listdir(dirs)
names = []
gradyears = []
for f in files:
    with open(dirs+f, 'r') as file:
        contents = file.read()
        soup = BeautifulSoup(contents, 'lxml')
        education = soup.find('h2', text='Education').find_next_sibling()
        s = education.text
        year_str_list = pattern.findall(s)
        year_int_list = list(map(int, year_str_list))
        if len(year_int_list) == 0:
            gradyears.append('')
        else:
            gradyears.append(min(year_int_list))
        name = os.path.splitext(f)[0]
        names.append(name)
df = pd.DataFrame({'name': names,
                   'gradyear': gradyears
                   })
df.to_csv('faculty_grad_year_test.csv')

### (c) Interesting Finding

Similar to Task 1(c), you don't need to do anything here. Just look at different visualizations w.r.t. age and give yourself an answer to the question: `Does SFU CS faculty age follow a normal distribution?`

In [2]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_grad_year.csv")
df["age"] = 2020+23-df["gradyear"]

plot(df, "age")

## Submission

Complete the code in this [notebook](https://github.com/sfu-db/bigdata-cmpt733/blob/master/Assignments/A1/A1.ipynb), and submit it to the CourSys activity `Assignment 1`.

<hr/>
## <span style="color:red">Bonus: Contribute to dataprep</span>

**Total Bonus: 0.2 + 0.3 = 0.5**

1. If you create an issue (i.e., report a bug or request a feature) at [link1](https://github.com/sfu-db/dataprep/issues) or [link2](https://github.com/sfu-db/DataConnectorConfigs/issues), you will get **0.2** bonus points.

2.  Creating more issues will *not* give you more bonus points, but you are encouraged to do so for learning more.

2. If you send a pull request (i.e., fix a bug or implement a feature or add data connector for a new website) at [link1](https://github.com/sfu-db/dataprep/pulls) or [link2](https://github.com/sfu-db/DataConnectorConfigs/pulls) and the pull request gets merged into the repo, you will get **0.3** bonus points. 

4.  Sending more pull requests will *not* give you more bonus points, but you are encouraged to do so for learning more.

5. These bonus points will be directly added to your final grade.






**How to Submit**
* Submit github link(s) to the CourSys activity `Bonus`.
* Due on `March 15, 2020`

If you love dataprep, please support it by simply clicking the **<span style="color:red">Star</span>** on [Github](https://github.com/sfu-db/dataprep).
<img src="dataprep-star.png" width="1000">